In [176]:
using JuMP
using Gurobi
using JSON
using Dates
const MOI = JuMP.MOI

# =================================================================================
# Gurobi license
# =================================================================================

function setup_gurobi_license()
    wls_access_id = get(ENV, "WLSACCESSID", "")
    wls_secret    = get(ENV, "WLSSECRET", "")
    license_id    = get(ENV, "LICENSEID", "")
    
    if !isempty(wls_access_id) && !isempty(wls_secret)
        println("🔑 Using Gurobi license from environment variables")
        ENV["GRB_WLSACCESSID"] = wls_access_id
        ENV["GRB_WLSSECRET"]   = wls_secret
        if !isempty(license_id)
            ENV["GRB_LICENSEID"] = license_id
        end
        return
    end
    
    license_file = joinpath(pwd(), "gurobi.lic")
    if isfile(license_file)
        println("🔑 Reading Gurobi license from $license_file")
        for line in eachline(license_file)
            line = strip(line)
            if !isempty(line) && !startswith(line, "#") && occursin("=", line)
                parts = split(line, "=", limit=2)
                if length(parts) == 2
                    key = strip(parts[1])
                    val = strip(parts[2])
                    ENV[key] = val
                    ENV["GRB_" * key] = val
                end
            end
        end
        println("✅ Gurobi license configured")
    else
        println("⚠️ No Gurobi license file found")
    end
end

# =================================================================================
# INPUT PARSING
# =================================================================================

function calculate_sessions(course, num_weeks, term_config)
    ctype      = get(course, "type", "full_term")
    period_min = term_config["period_length_minutes"]
    
    if ctype == "first_half_term"
        w_start = 1
        w_end   = div(num_weeks, 2)
        req_min = 3.0 * 60       # double course
    elseif ctype == "second_half_term"
        w_start = div(num_weeks, 2) + 1
        w_end   = num_weeks
        req_min = 3.0 * 60
    else
        w_start = 1
        w_end   = num_weeks
        req_min = 1.5 * 60       # single
    end
    
    periods         = Int(ceil(req_min / period_min))
    active_weeks    = w_end - w_start + 1
    total_sessions  = active_weeks
    
    return (periods_per_session = periods,
            total_sessions      = total_sessions,
            week_start          = w_start,
            week_end            = w_end)
end

function get_lunch_periods(term_config, num_periods)
    day_start = term_config["day_start_time"]
    p_len     = term_config["period_length_minutes"]
    
    s_parts   = split(day_start, ":")
    start_min = parse(Int, s_parts[1]) * 60 + parse(Int, s_parts[2])
    
    lunch_start_min = 12 * 60
    lunch_end_min   = 12 * 60 + 30
    
    lunch_periods = Int[]
    for p in 1:num_periods
        p_start = start_min + (p-1)*p_len
        p_end   = p_start + p_len
        if max(p_start, lunch_start_min) < min(p_end, lunch_end_min)
            push!(lunch_periods, p)
        end
    end
    return lunch_periods
end

function parse_input(input_dict::Dict)
    term_config      = input_dict["term_config"]
    classrooms       = input_dict["classrooms"]
    instructors      = input_dict["instructors"]
    courses          = input_dict["courses"]
    students         = input_dict["students"]
    conflict_weights = input_dict["conflict_weights"]
    
    num_weeks = term_config["num_weeks"]
    days      = String[d for d in term_config["days"]]
    
    start_time     = term_config["day_start_time"]
    end_time       = term_config["day_end_time"]
    period_minutes = term_config["period_length_minutes"]
    
    s_parts = split(start_time, ":")
    e_parts = split(end_time,   ":")
    start_min = parse(Int, s_parts[1]) * 60 + parse(Int, s_parts[2])
    end_min   = parse(Int, e_parts[1]) * 60 + parse(Int, e_parts[2])
    day_mins  = end_min - start_min
    num_periods = div(day_mins, period_minutes)
    
    room_ids = String[r["id"] for r in classrooms]
    room_cap = Int[r["capacity"] for r in classrooms]
    
    inst_ids = String[i["id"] for i in instructors]
    
    # Availability & b2b preference
    avail = fill(true, length(instructors), length(days), num_periods)
    inst_b2b_pref = Int[get(i, "back_to_back_preference", 0) for i in instructors]  # 1 dislike, 0 neutral, -1 like
    
    for (i, inst) in enumerate(instructors)
        if haskey(inst, "availability") && !isempty(inst["availability"])
            avail[i, :, :] .= false
            for slot in inst["availability"]
                d_idx = findfirst(==(slot["day"]), days)
                p_idx = slot["period_index"] + 1
                if d_idx !== nothing && 1 <= p_idx <= num_periods
                    avail[i, d_idx, p_idx] = true
                end
            end
        end
    end
    
    # Lunch teaching preference
    inst_lunch_penalty = Float64[
        get(inst, "allow_lunch_teaching", true) ? 0.0 : 1.0
        for inst in instructors
    ]
    
    # Courses
    course_ids   = String[c["id"] for c in courses]
    course_inst  = Int[]
    course_enr   = Int[c["expected_enrollment"] for c in courses]
    for c in courses
        idx = findfirst(id -> id == c["instructor_id"], inst_ids)
        push!(course_inst, idx)
    end
    
    session_info        = [calculate_sessions(c, num_weeks, term_config) for c in courses]
    total_sessions      = [s.total_sessions for s in session_info]
    periods_per_session = [s.periods_per_session for s in session_info]
    week_starts         = [s.week_start for s in session_info]
    week_ends           = [s.week_end   for s in session_info]
    
    # Student conflicts
    students_cc = zeros(Int, length(courses), length(courses))
    c_id_map    = Dict(id => i for (i, id) in enumerate(course_ids))
    for s in students
        enrolled = [c_id_map[cid] for cid in s["enrolled_course_ids"] if haskey(c_id_map, cid)]
        for i in 1:length(enrolled), j in (i+1):length(enrolled)
            c1, c2 = enrolled[i], enrolled[j]
            students_cc[c1, c2] += 1
            students_cc[c2, c1] += 1
        end
    end
    
    w1 = Float64(get(conflict_weights, "global_student_conflict_weight",      1.0))
    w2 = Float64(get(conflict_weights, "instructor_compactness_weight",       1.0))
    w3 = Float64(get(conflict_weights, "preferred_time_slots_weight",         1.0))
    
    return (
        num_weeks          = num_weeks,
        days               = days,
        num_periods        = num_periods,
        room_ids           = room_ids,
        room_cap           = room_cap,
        inst_ids           = inst_ids,
        avail              = avail,
        inst_b2b_pref      = inst_b2b_pref,
        course_ids         = course_ids,
        course_inst        = course_inst,
        course_enr         = course_enr,
        total_sessions     = total_sessions,
        periods_per_session= periods_per_session,
        week_starts        = week_starts,
        week_ends          = week_ends,
        students_cc        = students_cc,
        w1                 = w1,
        w2                 = w2,
        w3                 = w3,
        inst_lunch_penalty = inst_lunch_penalty,
        courses            = courses,
        classrooms         = classrooms,
        instructors        = instructors,
        term_config        = term_config
    )
end

# =================================================================================
# MODEL BUILDER
# =================================================================================

function build_and_solve_model(parsed)
    C = length(parsed.course_ids)
    I = length(parsed.inst_ids)
    R = length(parsed.room_ids)
    W = parsed.num_weeks
    D = length(parsed.days)
    P = parsed.num_periods
    
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 300)
    
    println("🏗️ Building Model: $C courses, $W weeks, $(length(parsed.days)) days, $P periods")
    
    # Decision variables
    @variable(model, x[1:C, 1:W, 1:D, 1:P, 1:R], Bin)  # course start
    @variable(model, h[1:I, 1:W, 1:D, 1:P], Bin)       # instructor teaching per period
    
    # Conflict variables (S1)
    @variable(model, φ[c1 in 1:C, c2 in (c1+1):C, w in 1:W, d in 1:D, p in 1:P], Bin)
    
    # Lunch penalty (S3)
    @variable(model, π[1:C, 1:W, 1:D, 1:P], Bin)
    
    # Back-to-back structure
    # z[i,c1,c2,w,d,p] = 1 if c1 starts at p and c2 starts immediately after c1
    @variable(model, z[1:I, 1:C, 1:C, 1:W, 1:D, 1:P], Bin)
    # b2b_sess[i,c1,c2,w,d] = 1 if there exists any p with z=1 (we use c1 < c2 only)
    @variable(model, b2b_sess[1:I, 1:C, 1:C, 1:W, 1:D], Bin)
    # teach_c[i,c,w,d] = 1 if instructor i teaches course c that day
    @variable(model, teach_c[1:I, 1:C, 1:W, 1:D], Bin)
    
    # Helper: valid start periods for course c that occupy period p
    function valid_starts(c, current_p)
        dur = parsed.periods_per_session[c]
        return max(1, current_p - dur + 1):min(current_p, P)
    end
    
    # --------------------------- Hard constraints ---------------------------
    
    # C1: Teacher conflict
    for i in 1:I, w in 1:W, d in 1:D, p in 1:P
        my_courses = [c for c in 1:C if parsed.course_inst[c] == i]
        occ = @expression(model,
            sum(x[c,w,d,s,r]
                for c in my_courses, r in 1:R, s in valid_starts(c,p)
                if s + parsed.periods_per_session[c] - 1 <= P)
        )
        @constraint(model, occ <= 1)
        @constraint(model, h[i,w,d,p] == occ)
    end
    
    # C2: Classroom conflict
    for r in 1:R, w in 1:W, d in 1:D, p in 1:P
        @constraint(model,
            sum(x[c,w,d,s,r]
                for c in 1:C, s in valid_starts(c,p)
                if s + parsed.periods_per_session[c] - 1 <= P) <= 1
        )
    end
    
    # C3: required sessions, duration & active weeks
    for c in 1:C
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]
        dur     = parsed.periods_per_session[c]
        req_s   = parsed.total_sessions[c]
        
        @constraint(model,
            sum(x[c,w,d,p,r]
                for w in w_start:w_end, d in 1:D, p in 1:P, r in 1:R
                if p + dur - 1 <= P) == req_s
        )
        
        all_weeks      = 1:W
        inactive_weeks = setdiff(all_weeks, w_start:w_end)
        for w in inactive_weeks
            @constraint(model,
                sum(x[c,w,d,p,r] for d in 1:D, p in 1:P, r in 1:R) == 0
            )
        end
    end
    
    # C4: availability
    for c in 1:C
        inst = parsed.course_inst[c]
        dur  = parsed.periods_per_session[c]
        for w in 1:W, d in 1:D, p in 1:P, r in 1:R
            if p + dur - 1 <= P
                is_avail = all(parsed.avail[inst,d,t] for t in p:(p+dur-1))
                if !is_avail
                    @constraint(model, x[c,w,d,p,r] == 0)
                end
            else
                @constraint(model, x[c,w,d,p,r] == 0)
            end
        end
    end
    
    # C7: capacity
    for c in 1:C, w in 1:W, d in 1:D, p in 1:P, r in 1:R
        @constraint(model, parsed.course_enr[c] * x[c,w,d,p,r] <= parsed.room_cap[r])
    end
    
    # C8: at most one session per day per course
    for c in 1:C, w in 1:W, d in 1:D
        dur = parsed.periods_per_session[c]
        @constraint(model,
            sum(x[c,w,d,p,r] for p in 1:P, r in 1:R if p + dur - 1 <= P) <= 1
        )
    end
    
    # C9: pattern consistency across weeks
    for c in 1:C
        w_start = parsed.week_starts[c]
        w_end   = parsed.week_ends[c]
        if w_end > w_start
            for w in (w_start+1):w_end, d in 1:D, p in 1:P, r in 1:R
                @constraint(model, x[c,w,d,p,r] == x[c,w_start,d,p,r])
            end
        end
    end
    
    # --------------------------- Soft constraints ---------------------------
    
    # S1: student conflicts
    for c1 in 1:C, c2 in (c1+1):C
        if parsed.students_cc[c1,c2] > 0
            for w in 1:W, d in 1:D, p in 1:P
                occ1 = @expression(model,
                    sum(x[c1,w,d,s,r] for r in 1:R, s in valid_starts(c1,p)
                        if s + parsed.periods_per_session[c1] - 1 <= P)
                )
                occ2 = @expression(model,
                    sum(x[c2,w,d,s,r] for r in 1:R, s in valid_starts(c2,p)
                        if s + parsed.periods_per_session[c2] - 1 <= P)
                )
                @constraint(model, occ1 + occ2 <= 1 + φ[c1,c2,w,d,p])
            end
        end
    end
    
    # ---------- Back-to-back structure (S2) ----------
    
    # z & b2b_sess for unordered pairs c1 < c2
    for i in 1:I
        for c1 in 1:(C-1), c2 in (c1+1):C
            if parsed.course_inst[c1] != i || parsed.course_inst[c2] != i
                continue
            end
            len1 = parsed.periods_per_session[c1]
            for w in 1:W, d in 1:D
                for p in 1:(P-len1)
                    @constraint(model,
                        z[i,c1,c2,w,d,p] <= sum(x[c1,w,d,p,r] for r in 1:R)
                    )
                    @constraint(model,
                        z[i,c1,c2,w,d,p] <= sum(x[c2,w,d,p+len1,r] for r in 1:R)
                    )
                    @constraint(model,
                        z[i,c1,c2,w,d,p] >=
                            sum(x[c1,w,d,p,r]      for r in 1:R) +
                            sum(x[c2,w,d,p+len1,r] for r in 1:R) - 1
                    )
                end
            end
        end
    end
    
    # b2b_sess is OR of z over p
    for i in 1:I
        for c1 in 1:(C-1), c2 in (c1+1):C
            if parsed.course_inst[c1] != i || parsed.course_inst[c2] != i
                continue
            end
            len1 = parsed.periods_per_session[c1]
            for w in 1:W, d in 1:D
                @constraint(model,
                    b2b_sess[i,c1,c2,w,d] <=
                        sum(z[i,c1,c2,w,d,p] for p in 1:(P-len1))
                )
                for p in 1:(P-len1)
                    @constraint(model,
                        z[i,c1,c2,w,d,p] <= b2b_sess[i,c1,c2,w,d]
                    )
                end
            end
        end
    end
    
    # teach_c: whether instructor teaches course c that day
    for i in 1:I, c in 1:C, w in 1:W, d in 1:D
        if parsed.course_inst[c] != i
            @constraint(model, teach_c[i,c,w,d] == 0)
        else
            total = @expression(model,
                sum(x[c,w,d,p,r] for p in 1:P, r in 1:R)
            )
            @constraint(model, teach_c[i,c,w,d] <= total)
            @constraint(model, teach_c[i,c,w,d] >= total / P)
        end
    end
    
    # S3: lunch periods
    lunch_periods = get_lunch_periods(parsed.term_config, P)
    for c in 1:C, w in 1:W, d in 1:D, p in lunch_periods
        occ = @expression(model,
            sum(x[c,w,d,s,r] for r in 1:R, s in valid_starts(c,p)
                if s + parsed.periods_per_session[c] - 1 <= P)
        )
        @constraint(model, occ <= π[c,w,d,p])
    end
    
    # --------------------------- Objective ---------------------------
    
    # S1: student conflicts
    obj_s1 = @expression(model,
        sum(parsed.w1 * parsed.students_cc[c1,c2] * φ[c1,c2,w,d,p]
            for c1 in 1:C, c2 in (c1+1):C, w in 1:W, d in 1:D, p in 1:P
            if parsed.students_cc[c1,c2] > 0)
    )
    
    # S2: symmetric back-to-back preference (per instructor/day)
    obj_s2_terms = Any[]
    for i in 1:I, w in 1:W, d in 1:D
        pref = parsed.inst_b2b_pref[i]
        if pref == 0
            continue
        end
        # Tᵢwd = #courses taught that day
        T_expr = @expression(model,
            sum(teach_c[i,c,w,d] for c in 1:C if parsed.course_inst[c] == i)
        )
        # Bᵢwd = #b2b unordered pairs that day
        B_expr = @expression(model,
            sum(b2b_sess[i,c1,c2,w,d]
                for c1 in 1:(C-1), c2 in (c1+1):C
                if parsed.course_inst[c1] == i && parsed.course_inst[c2] == i)
        )
        # S2_{iwd} = w2 * pref_i * (2 B - (T - 1))
        push!(obj_s2_terms, parsed.w2 * pref * (2*B_expr - (T_expr - 1)))
    end
    obj_s2 = @expression(model, sum(obj_s2_terms))
    
    # S3: lunch penalty
    obj_s3 = @expression(model,
        sum(parsed.w3 *
            parsed.inst_lunch_penalty[parsed.course_inst[c]] *
            π[c,w,d,p]
            for c in 1:C, w in 1:W, d in 1:D, p in lunch_periods)
    )
    
    @objective(model, Min, obj_s1 + obj_s2 + obj_s3)
    
    println("  Constraints: ",
        num_constraints(model, AffExpr, MOI.LessThan{Float64}) +
        num_constraints(model, AffExpr, MOI.EqualTo{Float64}))
    println("  Variables:   ", num_variables(model))
    println("  🚀 Optimizing...")
    
    optimize!(model)
    
    status = termination_status(model)
    println("  Status:     $status")
    println("  Solve time: $(round(solve_time(model), digits=3))s")
    
    println("=== Objective decomposition ===")
    println("  S1_student_conflicts = ", value(obj_s1))
    println("  S2_back_to_back      = ", value(obj_s2))
    println("  S3_lunch_penalty     = ", value(obj_s3))
    println("  Total objective      = ", objective_value(model))
    
    # ---------- DEBUG: S2 details ----------
    println("=== S2 DEBUG BY INSTRUCTOR / DAY ===")
    vals_teach_c   = value.(teach_c)
    vals_b2b_sess  = value.(b2b_sess)
    for i in 1:I
        pref = parsed.inst_b2b_pref[i]
        if pref == 0
            continue
        end
        println("Instructor $(parsed.inst_ids[i]) pref = $pref (−1 likes b2b, +1 dislikes b2b)")
        for w in 1:W, d in 1:D
            # T = #courses taught that day
            T = sum(vals_teach_c[i,c,w,d] for c in 1:C if parsed.course_inst[c] == i)
            if T < 0.5
                continue
            end
            B = 0.0
            for c1 in 1:(C-1), c2 in (c1+1):C
                if parsed.course_inst[c1] == i && parsed.course_inst[c2] == i
                    B += vals_b2b_sess[i,c1,c2,w,d]
                end
            end
            E = T - 1.0
            contrib = parsed.w2 * pref * (2B - (T - 1.0))
            println("  Week $w, Day $(parsed.days[d]): T=$T, edges(E)=T-1=$E, B=$B, contrib=$contrib")
        end
    end
    
    println("=== Assignments ===")
    vals_x = value.(x)
    for c in 1:C, w in 1:W, d in 1:D, p in 1:P, r in 1:R
        if vals_x[c,w,d,p,r] > 0.5
            println("Course $(parsed.course_ids[c]) in room $(parsed.room_ids[r])",
                    " | week=$w day=$(parsed.days[d]) period_start=$p",
                    " (len=$(parsed.periods_per_session[c]))")
        end
    end
    
    return model, x
end

# =================================================================================
# OUTPUT FORMATTERS
# =================================================================================

function format_optimal_solution(model, x, parsed)
    assignments = []
    vals = value.(x)
    C,W,D,P,R = size(vals)
    
    session_counters = Dict{Int,Int}()
    for c in 1:C
        session_counters[c] = 0
    end
    
    for c in 1:C, w in 1:W, d in 1:D, p in 1:P, r in 1:R
        if vals[c,w,d,p,r] > 0.5
            session_counters[c] += 1
            snum = session_counters[c]
            course = parsed.courses[c]
            room   = parsed.classrooms[r]
            inst_idx = parsed.course_inst[c]
            inst     = parsed.instructors[inst_idx]
            push!(assignments, Dict(
                "course_id"         => parsed.course_ids[c],
                "course_session_id" => "$(parsed.course_ids[c])_S$(snum)",
                "session_number"    => snum,
                "course_name"       => get(course, "name", ""),
                "room_id"           => parsed.room_ids[r],
                "room_name"         => get(room, "name", ""),
                "week"              => w - 1,
                "day"               => parsed.days[d],
                "period_start"      => p - 1,
                "period_length"     => parsed.periods_per_session[c],
                "instructor_id"     => get(inst, "id", ""),
                "instructor_name"   => get(inst, "name", "")
            ))
        end
    end
    
    obj_val = has_values(model) ? objective_value(model) : nothing
    
    return Dict(
        "status"             => termination_status(model) == MOI.OPTIMAL ? "optimal" : "time_limit",
        "objective_value"    => obj_val,
        "solve_time_seconds" => solve_time(model),
        "hard_constraints_ok"=> true,
        "violated_hard_constraints" => String[],
        "soft_constraint_summary" => Dict(
            "S1_student_conflicts" => Dict(
                "weighted_penalty" => obj_val,
                "weight" => parsed.w1,
                "details"=> "Student schedule conflicts (weight: $(parsed.w1))"
            ),
            "S2_instructor_compactness" => Dict(
                "weighted_penalty" => 0.0,
                "weight" => parsed.w2,
                "details"=> "Instructor back-to-back preference (weight: $(parsed.w2))"
            ),
            "S3_preferred_time_slots" => Dict(
                "weighted_penalty" => 0.0,
                "weight" => parsed.w3,
                "details"=> "Lunch period penalties (weight: $(parsed.w3))"
            )
        ),
        "schedule" => Dict("assignments" => assignments),
        "diagnostics" => Dict(
            "room_capacity_violations" => [],
            "teacher_overlaps"         => [],
            "student_conflicts"        => [],
            "lunch_violations"         => []
        ),
        "metadata" => Dict(
            "solver"                     => "Gurobi (Julia/JuMP)",
            "constraint_metadata_version"=> "1.0",
            "num_assignments"            => length(assignments)
        )
    )
end

function format_infeasible_solution(model, parsed)
    try
        compute_conflict!(model)
        println("  IIS computation attempted")
    catch
        println("  IIS computation failed")
    end
    return Dict(
        "status"             => "infeasible",
        "objective_value"    => nothing,
        "solve_time_seconds" => solve_time(model),
        "hard_constraints_ok"=> false,
        "violated_hard_constraints" => ["multiple_constraints"],
        "soft_constraint_summary"   => Dict(),
        "schedule" => Dict("assignments" => []),
        "diagnostics" => Dict(
            "infeasibility_explanation" => "No feasible schedule satisfies all hard constraints"
        ),
        "metadata" => Dict(
            "solver"                     => "Gurobi (Julia/JuMP)",
            "constraint_metadata_version"=> "1.0"
        )
    )
end

function format_output(model, x, parsed)
    status = termination_status(model)
    if status in (MOI.INFEASIBLE, MOI.INFEASIBLE_OR_UNBOUNDED)
        return format_infeasible_solution(model, parsed)
    elseif status != MOI.OPTIMAL && status != MOI.TIME_LIMIT
        return Dict(
            "status"             => string(status),
            "objective_value"    => nothing,
            "solve_time_seconds" => solve_time(model),
            "hard_constraints_ok"=> false,
            "violated_hard_constraints" => [],
            "soft_constraint_summary"   => Dict(),
            "schedule" => Dict("assignments" => []),
            "diagnostics" => Dict(
                "gurobi_status" => string(status),
                "message"       => "Solver terminated with status: $(string(status))"
            ),
            "metadata" => Dict(
                "solver" => "Gurobi (Julia/JuMP)"
            )
        )
    else
        return format_optimal_solution(model, x, parsed)
    end
end

# =================================================================================
# TOP-LEVEL SOLVER
# =================================================================================

function solve_scheduling_problem(input_dict::Dict)
    try
        println("📊 Julia solver started at $(now())")
        setup_gurobi_license()
        parsed        = parse_input(input_dict)
        model, x      = build_and_solve_model(parsed)
        output        = format_output(model, x, parsed)
        println("✅ Julia solver completed: $(output["status"])")
        return output
    catch e
        msg = string(e)
        println("❌ Julia solver error: $msg")
        tb = try
            sprint(showerror, e, catch_backtrace())
        catch
            "Stack trace unavailable"
        end
        println("   Traceback: $tb")
        return Dict(
            "status"             => "error",
            "objective_value"    => nothing,
            "solve_time_seconds" => 0.0,
            "hard_constraints_ok"=> false,
            "violated_hard_constraints" => [],
            "soft_constraint_summary"   => Dict(),
            "schedule" => Dict("assignments" => []),
            "diagnostics" => Dict(
                "error"     => msg,
                "traceback" => tb
            ),
            "metadata" => Dict(
                "solver"   => "Gurobi (Julia/JuMP)",
                "timestamp"=> string(now())
            )
        )
    end
end

solve_scheduling_problem (generic function with 1 method)

In [178]:
using JSON

# =================================================================================
# SIMPLE TEST: Verify symmetric back-to-back rewards
# Expected: reward_likes_b2b ≈ reward_dislikes_b2b (same magnitude, both negative)
# =================================================================================

"""
Create minimal test input
"""
function make_test_input(pref::Int, num_courses::Int=2)
    return Dict{String,Any}(
        "term_config" => Dict{String,Any}(
            "num_weeks" => 10,
            "days" => ["Mon"],
            "period_length_minutes" => 30,
            "day_start_time" => "08:00",
            "day_end_time" => "20:00"
        ),
        "classrooms" => [
            Dict("id" => "R1", "name" => "Room1", "capacity" => 100)
        ],
        "instructors" => [
            Dict(
                "id" => "Prof1",
                "name" => "Professor 1",
                "back_to_back_preference" => pref,  # -1 likes, +1 dislikes
                "allow_lunch_teaching" => true
            )
        ],
        "courses" => [
            Dict(
                "id" => "C$i",
                "name" => "Course$i",
                "type" => "full_term",
                "weekly_hours" => 1.5,
                "instructor_id" => "Prof1",
                "expected_enrollment" => 50
            ) for i in 1:num_courses
        ],
        "students" => [],  # No student conflicts
        "conflict_weights" => Dict{String,Any}(
            "global_student_conflict_weight" => 1.0,
            "instructor_compactness_weight" => 1.0,
            "preferred_time_slots_weight" => 1.0
        )
    )
end

"""
Run test and print results
"""
function run_test(num_courses::Int=2)
    println("\n" * "="^70)
    println("TEST: $num_courses courses - Symmetric B2B Check")
    println("="^70)
    
    # Test 1: Instructor LIKES back-to-back (pref = -1)
    println("\n▶ Running: LIKES back-to-back (pref=-1)")
    input_likes = make_test_input(-1, num_courses)
    result_likes = solve_scheduling_problem(input_likes)
    
    obj_likes = result_likes["objective_value"]
    status_likes = result_likes["status"]
    println("  Status: $status_likes")
    println("  Objective: $obj_likes")
    
    # Test 2: Instructor DISLIKES back-to-back (pref = +1)
    println("\n▶ Running: DISLIKES back-to-back (pref=+1)")
    input_dislikes = make_test_input(1, num_courses)
    result_dislikes = solve_scheduling_problem(input_dislikes)
    
    obj_dislikes = result_dislikes["objective_value"]
    status_dislikes = result_dislikes["status"]
    println("  Status: $status_dislikes")
    println("  Objective: $obj_dislikes")
    
    # Check symmetry
    println("\n" * "-"^70)
    println("SYMMETRY CHECK:")
    println("-"^70)
    
    if obj_likes !== nothing && obj_dislikes !== nothing
        println("  Likes b2b objective:    $obj_likes")
        println("  Dislikes b2b objective: $obj_dislikes")
        
        # Both should have same value (both negative in this formula)
        diff = abs(obj_likes - obj_dislikes)
        
        if diff < 1e-4
            println("\n  ✅ PASS: Symmetric! (difference = $diff)")
            println("  Both rewards are equal: $obj_likes ≈ $obj_dislikes")
        else
            println("\n  ❌ FAIL: Not symmetric! (difference = $diff)")
        end
    else
        println("  ⚠️  Cannot verify - one or both solutions failed")
    end
    
    println("="^70 * "\n")
end

# =================================================================================
# RUN TESTS
# =================================================================================

println("\n🚀 Starting B2B Symmetry Tests")

# Test with 2 courses
run_test(2)



🚀 Starting B2B Symmetry Tests

TEST: 2 courses - Symmetric B2B Check

▶ Running: LIKES back-to-back (pref=-1)
📊 Julia solver started at 2025-12-01T15:19:54.714
⚠️ No Gurobi license file found
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2743592
Academic license 2743592 - for non-commercial use only - registered to er___@stanford.edu
🏗️ Building Model: 2 courses, 10 weeks, 1 days, 24 periods
  Constraints: 2374
  Variables:   2460
  🚀 Optimizing...
Set parameter TimeLimit to value 300
  Status:     OPTIMAL
  Solve time: 0.006s
=== Objective decomposition ===
  S1_student_conflicts = 0.0
  S2_back_to_back      = -10.0
  S3_lunch_penalty     = 0.0
  Total objective      = -10.0
=== S2 DEBUG BY INSTRUCTOR / DAY ===
Instructor Prof1 pref = -1 (−1 likes b2b, +1 dislikes b2b)
  Week 1, Day Mon: T=2.0, edges(E)=T-1=1.0, B=1.0, contrib=-1.0
  Week 2, Day Mon: T=2.0, edges(E)=T-1=1.0, B=1.0, contrib=-1.0
  Week 3, Day Mon: T=2.0, edges(E)=T-1=1.0, B=1.0, co

In [174]:
# Test with 3 courses
run_test(3)


TEST: 3 courses - Symmetric B2B Check

▶ Running: LIKES back-to-back (pref=-1)
📊 Julia solver started at 2025-12-01T15:18:52.521
⚠️ No Gurobi license file found
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2743592
Academic license 2743592 - for non-commercial use only - registered to er___@stanford.edu
🏗️ Building Model: 3 courses, 10 weeks, 1 days, 24 periods
  Constraints: 4161
  Variables:   4680
  🚀 Optimizing...
Set parameter TimeLimit to value 300
  Status:     OPTIMAL
  Solve time: 0.068s
=== Objective decomposition ===
  S1_student_conflicts = 0.0
  S2_back_to_back      = -20.0
  S3_lunch_penalty     = 0.0
  Total objective      = -20.0
=== S2 DEBUG BY INSTRUCTOR / DAY ===
Instructor Prof1 pref = -1 (−1 likes b2b, +1 dislikes b2b)
  Week 1, Day Mon: T=3.0, edges(E)=T-1=2.0, B=2.0, contrib=-2.0
  Week 2, Day Mon: T=3.0, edges(E)=T-1=2.0, B=2.0, contrib=-2.0
  Week 3, Day Mon: T=3.0, edges(E)=T-1=2.0, B=2.0, contrib=-2.0
  Week 4, Day Mon: T

In [175]:
# Test with 3 courses
run_test(4)


TEST: 4 courses - Symmetric B2B Check

▶ Running: LIKES back-to-back (pref=-1)
📊 Julia solver started at 2025-12-01T15:18:58.762
⚠️ No Gurobi license file found
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2743592
Academic license 2743592 - for non-commercial use only - registered to er___@stanford.edu
🏗️ Building Model: 4 courses, 10 weeks, 1 days, 24 periods
  Constraints: 6588
  Variables:   7640
  🚀 Optimizing...
Set parameter TimeLimit to value 300
  Status:     OPTIMAL
  Solve time: 0.241s
=== Objective decomposition ===
  S1_student_conflicts = 0.0
  S2_back_to_back      = -30.0
  S3_lunch_penalty     = 0.0
  Total objective      = -30.0
=== S2 DEBUG BY INSTRUCTOR / DAY ===
Instructor Prof1 pref = -1 (−1 likes b2b, +1 dislikes b2b)
  Week 1, Day Mon: T=4.0, edges(E)=T-1=3.0, B=3.0, contrib=-3.0
  Week 2, Day Mon: T=4.0, edges(E)=T-1=3.0, B=3.0, contrib=-3.0
  Week 3, Day Mon: T=4.0, edges(E)=T-1=3.0, B=3.0, contrib=-3.0
  Week 4, Day Mon: T